<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/APACHEAGE_and_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://age.apache.org/age-manual/master/intro/overview.html
# https://cookbook.openai.com/examples/rag_with_graph_db
#viewer https://github.com/apache/age-viewer
# https://github.com/apache/age

!apt-get update
!apt-get dist-upgrade
!sudo apt-get install build-essential libreadline-dev zlib1g-dev flex bison

# https://cookbook.openai.com/examples/rag_with_graph_db

!pip install langchain --quiet
!pip install openai --quiet
!pip install neo4j --quiet

!pip install colab-env --upgrade
!pip install openai
!pip install tiktoken

In [ ]:
#!sudo apt install postgresql

#ADDED By FM 11/01/2024

# install PSQL WITH DEV Libraries
!apt install postgresql postgresql-contrib &>log
!service postgresql restart
!sudo apt install postgresql-server-dev-all


In [3]:
print()
# PostGRES SQL Settings
%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"


/content
ALTER ROLE


In [ ]:

%cd /content/
#release/PG14/1.5.0
!git clone --branch release/PG14/1.5.0 https://github.com/apache/age.git

print('START: A Graph Extension(AGE) COMPILATION')
%cd /content/age
print()
!pg_config
print()
!make install # may need sudo
print('END: A Graph Extension(AGE) COMPILATION')
print()

In [5]:
!sudo -u postgres psql -c "CREATE EXTENSION age"
!sudo -u postgres psql -c "LOAD 'age'"
!sudo -u postgres psql -c 'SET search_path = ag_catalog, "$user", public'

#!sudo -u postgres psql -c "SELECT create_graph('my_graph_name')"

CREATE EXTENSION
LOAD
SET


In [ ]:
# https://github.com/apache/age/tree/master/drivers/python
%cd /content/age/drivers/python/
%pip install -r /content/age/drivers/python/requirements.txt
!python /content/age/drivers/python/setup.py install

In [7]:
#SELECT create_graph('my_graph_name');
import age

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

age.connect(graph='my_graph_name', host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
#cursor = age.execCypher("SELECT create_graph('my_graph_name')")
#age.execSql(stmt="CREATE GRAPH my_graph_name")
#age.execSql("create_graph('my_graph_name')")
#!sudo -u postgres psql -c "SELECT create_graph('my_graph_name')"

In [8]:
def query_selection(query):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""
          %s """%query)
      records = cur.fetchall()
      print("Total rows are:  ", len(records))
      print("Printing each row")
      print()
      n=0
      for row in records:
          n=n+1
          print("ROW %s: "%n, row)
      conn.close()
      cur.close()
      print()
      print("QUERY SELECTION successfully")
      print()

In [9]:
print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")



Extensions Available:
Total rows are:   47
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('age',)
ROW 3:  ('amcheck',)
ROW 4:  ('autoinc',)
ROW 5:  ('bloom',)
ROW 6:  ('btree_gin',)
ROW 7:  ('btree_gist',)
ROW 8:  ('citext',)
ROW 9:  ('cube',)
ROW 10:  ('dblink',)
ROW 11:  ('dict_int',)
ROW 12:  ('dict_xsyn',)
ROW 13:  ('earthdistance',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('sslinfo',)

In [10]:
!python -m unittest -v /content/age/drivers/python/test_agtypes.py

test_path (test_agtypes.TestAgtype) ... 
Testing Path Parsing. Result : ok
test_scalar (test_agtypes.TestAgtype) ... 
Testing Scalar Value Parsing. Result : ok
test_vertex (test_agtypes.TestAgtype) ... 
Testing vertex Parsing. Result : ok

----------------------------------------------------------------------
Ran 3 tests in 0.024s

OK


In [11]:
!python /content/age/drivers/python/test_age_py.py \
-host "127.0.0.1" \
-db "postgres" \
-u "postgres" \
-pass "postgres" \
-port 5432 \
-gn "test_graph"

Connecting to Test Graph.....

---------------------------------------------------
Test 1: Checking single and multi column Returns.....
---------------------------------------------------

Vertex: <class 'age.models.Vertex'> , Type: <class 'age.models.Vertex'> 
Id: 844424930131970 , Name: Jack

Test 1 Successful....
Deleting Test Graph.....
.Connecting to Test Graph.....

--------------------------------------------------
Test 2: Testing CREATE and query relationships.....
--------------------------------------------------

START: {label:Person, id:844424930131970, properties:{name: Andy, }}::VERTEX
EDGE: {label:worksWith, id:1125899906842625, properties:{weight: 3, }, start_id:844424930131970, end_id:844424930131969}::EDGE
END: {label:Person, id:844424930131969, properties:{name: Jack, }}::VERTEX
START: {label:Person, id:844424930131971, properties:{name: Smith, }}::VERTEX
EDGE: {label:worksWith, id:1125899906842626, properties:{}, start_id:844424930131969, end_id:844424930131971}::E

# NEO4J - Installation and Configurations

In [ ]:
# https://gist.github.com/korakot/328aaac51d78e589b4a176228e4bb06f#file-neo4j-py
#!wget https://neo4j.com/artifact.php?name=neo4j-community-4.3.0-unix.tar.gz
#https://we-yun.com/doc/neo4j/5.16.0/


%cd /content/
!wget https://neo4j.com/artifact.php?name=neo4j-community-5.16.0-unix.tar.gz
!tar -xzf /content/artifact.php?name=neo4j-community-5.16.0-unix.tar.gz
!sudo mv /content/neo4j-community-5.16.0  /usr/local/neo4j
%cd /content/

#!/usr/local/neo4j/bin/neo4j start

In [13]:
!sed -i '/dbms.security.auth_enabled/s/^#//g' /usr/local/neo4j/conf/neo4j.conf
!sed -i 's/#dbms.security.procedures.unrestricted=my.extensions.example,my.procedures.*/dbms.security.procedures.unrestricted=apoc.*/g' /usr/local/neo4j/conf/neo4j.conf
!sed -i 's/#dbms.security.procedures.allowlist=apoc.coll.*,apoc.load.*,gds.*/dbms.security.procedures.allowlist=apoc.*/g' /usr/local/neo4j/conf/neo4j.conf

In [ ]:
!sudo apt-get install openjdk-21-jre

In [15]:
!cp /usr/local/neo4j/labs/apoc-5.16.1-core.jar /usr/local/neo4j/plugins/

In [16]:
!/usr/local/neo4j/bin/neo4j start
#!/usr/local/neo4j/bin/neo4j-admin set-initial-password neo4j

Directories in use:
home:         /usr/local/neo4j
config:       /usr/local/neo4j/conf
logs:         /usr/local/neo4j/logs
plugins:      /usr/local/neo4j/plugins
import:       /usr/local/neo4j/import
data:         /usr/local/neo4j/data
certificates: /usr/local/neo4j/certificates
licenses:     /usr/local/neo4j/licenses
run:          /usr/local/neo4j/run
Starting Neo4j.
Started neo4j (pid:5059). It is available at http://localhost:7474
There may be a short delay until the server is ready.


In [17]:
!pip install neo4j --quiet
import neo4j
neo4j.__version__

'5.17.0'

In [ ]:
!pip install py2neo -q
from py2neo import Graph
graph = Graph("bolt://localhost:7687")

from IPython.core.magic import register_cell_magic
@register_cell_magic
def nj(line, cell=None):
  return graph.run(cell or line).to_table()

In [19]:
# DB credentials
url = "bolt://localhost:7687"
username ="neo4j"
password = "neo4j"

from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)



In [20]:
# https://cookbook.openai.com/examples/rag_with_graph_db
%cd /content/
!git clone https://github.com/openai/openai-cookbook.git

import json
# Loading a json dataset from a file
file_path = '/content/openai-cookbook/examples/data/amazon_product_kg.json'

with open(file_path, 'r') as file:
    jsonData = json.load(file)

import os
import json
import pandas as pd

df =  pd.read_json(file_path)
df.head()

/content
Cloning into 'openai-cookbook'...
remote: Enumerating objects: 4229, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 4229 (delta 0), reused 1 (delta 0), pack-reused 4223
Receiving objects: 100% (4229/4229), 211.29 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (2512/2512), done.
Updating files: 100% (271/271), done.


,product_id,product,relationship,entity_type,entity_value,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,Blackout Curtain,hasCategory,category,home decoration,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
1,1925202,Blackout Curtain,hasBrand,brand,ArtzFolio,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
2,1925202,Blackout Curtain,hasCharacteristic,characteristic,Eyelets,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
3,1925202,Blackout Curtain,hasCharacteristic,characteristic,Tie Back,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98
4,1925202,Blackout Curtain,hasCharacteristic,characteristic,100% opaque,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,None,1650,2125.98


In [21]:
# https://cookbook.openai.com/examples/rag_with_graph_db

def sanitize(text):
    text = str(text).replace("'","").replace('"','').replace('{','').replace('}', '')
    return text

# Loop through each JSON object and add them to the db
i = 1
for obj in jsonData:
    #print(f"{i}. {obj['product_id']} -{obj['relationship']}-> {obj['entity_value']}")
    i+=1
    query = f'''
        MERGE (product:Product {{id: {obj['product_id']}}})
        ON CREATE SET product.name = "{sanitize(obj['product'])}",
                       product.title = "{sanitize(obj['TITLE'])}",
                       product.bullet_points = "{sanitize(obj['BULLET_POINTS'])}",
                       product.size = {sanitize(obj['PRODUCT_LENGTH'])}

        MERGE (entity:{obj['entity_type']} {{value: "{sanitize(obj['entity_value'])}"}})

        MERGE (product)-[:{obj['relationship']}]->(entity)
        '''
    graph.query(query)

In [22]:
import colab_env
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

Mounted at /content/gdrive


In [23]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
#embeddings_model = "text-embedding-3-small"
embeddings_model = "text-embedding-ada-002"
#embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model=embeddings_model),
    url=url,
    username=username,
    password=password,
    index_name='products',
    node_label="Product",
    text_node_properties=['name', 'title'],
    embedding_node_property='embedding',
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [24]:
def embed_entities(entity_type):
    vector_index = Neo4jVector.from_existing_graph(
        OpenAIEmbeddings(model=embeddings_model),
        url=url,
        username=username,
        password=password,
        index_name=entity_type,
        node_label=entity_type,
        text_node_properties=['value'],
        embedding_node_property='embedding',
    )

entities_list = df['entity_type'].unique()

for t in entities_list:
    embed_entities(t)

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [26]:
chain.run("""
Help me find curtains
""")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)-[:HAS_CATEGORY]->(c:Category)
WHERE c.name = 'Curtains'
RETURN p
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have any information to help you find curtains."

In [27]:
entity_types = {
    "product": "Item detailed type, for example 'high waist pants', 'outdoor plant pot', 'chef kitchen knife'",
    "category": "Item category, for example 'home decoration', 'women clothing', 'office supply'",
    "characteristic": "if present, item characteristics, for example 'waterproof', 'adhesive', 'easy to use'",
    "measurement": "if present, dimensions of the item",
    "brand": "if present, brand of the item",
    "color": "if present, color of the item",
    "age_group": "target age group for the product, one of 'babies', 'children', 'teenagers', 'adults'. If suitable for multiple age groups, pick the oldest (latter in the list)."
}

relation_types = {
    "hasCategory": "item is of this category",
    "hasCharacteristic": "item has this characteristic",
    "hasMeasurement": "item is of this measurement",
    "hasBrand": "item is of this brand",
    "hasColor": "item is of this color",
    "isFor": "item is for this age_group"
 }

entity_relationship_match = {
    "category": "hasCategory",
    "characteristic": "hasCharacteristic",
    "measurement": "hasMeasurement",
    "brand": "hasBrand",
    "color": "hasColor",
    "age_group": "isFor"
}

In [28]:
system_prompt = f'''
    You are a helpful agent designed to fetch information from a graph database.

    The graph database links products to the following entity types:
    {json.dumps(entity_types)}

    Each link has one of the following relationships:
    {json.dumps(relation_types)}

    Depending on the user prompt, determine if it possible to answer with the graph database.

    The graph database can match products with multiple relationships to several entities.

    Example user input:
    "Which blue clothing items are suitable for adults?"

    There are three relationships to analyse:
    1. The mention of the blue color means we will search for a color similar to "blue"
    2. The mention of the clothing items means we will search for a category similar to "clothing"
    3. The mention of adults means we will search for an age_group similar to "adults"


    Return a json object following the following rules:
    For each relationship to analyse, add a key value pair with the key being an exact match for one of the entity types provided, and the value being the value relevant to the user query.

    For the example provided, the expected output would be:
    {{
        "color": "blue",
        "category": "clothing",
        "age_group": "adults"
    }}

    If there are no relevant entities in the user prompt, return an empty json object.
'''

print(system_prompt)


    You are a helpful agent designed to fetch information from a graph database.

    The graph database links products to the following entity types:
    {"product": "Item detailed type, for example 'high waist pants', 'outdoor plant pot', 'chef kitchen knife'", "category": "Item category, for example 'home decoration', 'women clothing', 'office supply'", "characteristic": "if present, item characteristics, for example 'waterproof', 'adhesive', 'easy to use'", "measurement": "if present, dimensions of the item", "brand": "if present, brand of the item", "color": "if present, color of the item", "age_group": "target age group for the product, one of 'babies', 'children', 'teenagers', 'adults'. If suitable for multiple age groups, pick the oldest (latter in the list)."}

    Each link has one of the following relationships:
    {"hasCategory": "item is of this category", "hasCharacteristic": "item has this characteristic", "hasMeasurement": "item is of this measurement", "hasBrand": "i

In [29]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

# Define the entities to look for
def define_query(prompt, model="gpt-4-1106-preview"):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format= {
            "type": "json_object"
        },
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return completion.choices[0].message.content

In [30]:
example_queries = [
    "Which pink items are suitable for children?",
    "Help me find gardening gear that is waterproof",
    "I'm looking for a bench with dimensions 100x50 for my living room"
]

for q in example_queries:
    print(f"Q: '{q}'\n{define_query(q)}\n")


Q: 'Which pink items are suitable for children?'
{
    "color": "pink",
    "age_group": "children"
}

Q: 'Help me find gardening gear that is waterproof'
{
    "category": "gardening gear",
    "characteristic": "waterproof"
}

Q: 'I'm looking for a bench with dimensions 100x50 for my living room'
{
    "product": "bench",
    "measurement": "100x50",
    "category": "living room"
}



In [31]:
def create_embedding(text):
    result = client.embeddings.create(model=embeddings_model, input=text)
    return result.data[0].embedding

In [32]:
# The threshold defines how closely related words should be. Adjust the threshold to return more or less results
def create_query(text, threshold=0.81):
    query_data = json.loads(text)
    # Creating embeddings
    embeddings_data = []
    for key, val in query_data.items():
        if key != 'product':
            embeddings_data.append(f"${key}Embedding AS {key}Embedding")
    query = "WITH " + ",\n".join(e for e in embeddings_data)
    # Matching products to each entity
    query += "\nMATCH (p:Product)\nMATCH "
    match_data = []
    for key, val in query_data.items():
        if key != 'product':
            relationship = entity_relationship_match[key]
            match_data.append(f"(p)-[:{relationship}]->({key}Var:{key})")
    query += ",\n".join(e for e in match_data)
    similarity_data = []
    for key, val in query_data.items():
        if key != 'product':
            similarity_data.append(f"gds.similarity.cosine({key}Var.embedding, ${key}Embedding) > {threshold}")
    query += "\nWHERE "
    query += " AND ".join(e for e in similarity_data)
    query += "\nRETURN p"
    return query

In [33]:
def query_graph(response):
    embeddingsParams = {}
    query = create_query(response)
    query_data = json.loads(response)
    for key, val in query_data.items():
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    result = graph.query(query, params=embeddingsParams)
    return result



In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


def query_graph(response):
    query_data = json.loads(response)
    embeddingsParams = {}
    for key, val in query_data.items():
        #embeddingsParams[f"{key}Embedding"] = embeddings.embed(val)
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    query = f"""
    MATCH (categoryVar:Category {{name: $categoryEmbedding}})
    MATCH (colorVar:Color {{name: $colorEmbedding}})
    MATCH (age_groupVar:AgeGroup {{name: $age_groupEmbedding}})
    RETURN categoryVar.name AS category, colorVar.name AS color, age_groupVar.name AS age_group
    """
    result = graph.query(query, params=embeddingsParams)
    return result

#graph = Neo4jGraph(embeddings=embeddings, db_type="bolt", uri="neo4j://localhost:7687", auth=("neo4j", "password"))

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

example_response = '''{
    "category": "clothes",
    "color": "blue",
    "age_group": "adults"
}'''

result = query_graph(example_response)

print(result)


In [ ]:
example_response = '''{
    "category": "clothes",
    "color": "blue",
    "age_group": "adults"
}'''

result = query_graph(example_response)
print(result)

In [ ]:
# Result
print(f"Found {len(result)} matching product(s):\n")
for r in result:
    print(f"{r['p']['name']} ({r['p']['id']})")

In [37]:
# Adjust the relationships_threshold to return products that have more or less relationships in common
def query_similar_items(product_id, relationships_threshold = 3):

    similar_items = []

    # Fetching items in the same category with at least 1 other entity in common
    query_category = '''
            MATCH (p:Product {id: $product_id})-[:hasCategory]->(c:category)
            MATCH (p)-->(entity)
            WHERE NOT entity:category
            MATCH (n:Product)-[:hasCategory]->(c)
            MATCH (n)-->(commonEntity)
            WHERE commonEntity = entity AND p.id <> n.id
            RETURN DISTINCT n;
        '''


    result_category = graph.query(query_category, params={"product_id": int(product_id)})
    #print(f"{len(result_category)} similar items of the same category were found.")

    # Fetching items with at least n (= relationships_threshold) entities in common
    query_common_entities = '''
        MATCH (p:Product {id: $product_id})-->(entity),
            (n:Product)-->(entity)
            WHERE p.id <> n.id
            WITH n, COUNT(DISTINCT entity) AS commonEntities
            WHERE commonEntities >= $threshold
            RETURN n;
        '''
    result_common_entities = graph.query(query_common_entities, params={"product_id": int(product_id), "threshold": relationships_threshold})
    #print(f"{len(result_common_entities)} items with at least {relationships_threshold} things in common were found.")

    for i in result_category:
        similar_items.append({
            "id": i['n']['id'],
            "name": i['n']['name']
        })

    for i in result_common_entities:
        result_id = i['n']['id']
        if not any(item['id'] == result_id for item in similar_items):
            similar_items.append({
                "id": result_id,
                "name": i['n']['name']
            })
    return similar_items

In [38]:
product_ids = ['1519827', '2763742']

for product_id in product_ids:
    print(f"Similar items for product #{product_id}:\n")
    result = query_similar_items(product_id)
    print("\n")
    for r in result:
        print(f"{r['name']} ({r['id']})")
    print("\n\n")



Similar items for product #1519827:



Womens Shift Knee-Long Dress (1483279)
Maxi Dresses (1818763)
Lingerie for Women for Sex Naughty (2666747)
Alpine Faux Suede Knit Pencil Skirt (1372443)
V-Neck Long Jumpsuit (2838428)
Womens Maroon Round Neck Full Sleeves Gathered Peplum Top (1256928)
Dhoti Pants (2293307)
Sun Uv Protection Driving Gloves (1844637)
Glossies Thong (941830)
Womens Lightly Padded Non-Wired Printed T-Shirt Bra (1954205)
Chiffon printed dupatta (2919319)
Underwire Bra (1325580)
Womens Drawstring Harem Pants (1233616)
Womens Satin Semi-Stitched Lehenga Choli (2763742)
Turtleneck Oversized Sweaters (2535064)
A Line Open Back Satin Prom Dress (1955999)
Womens Cotton Ankle Length Leggings (1594019)



Similar items for product #2763742:



Womens Shift Knee-Long Dress (1483279)
Maxi Dresses (1818763)
Lingerie for Women for Sex Naughty (2666747)
Alpine Faux Suede Knit Pencil Skirt (1372443)
V-Neck Long Jumpsuit (2838428)
Womens Maroon Round Neck Full Sleeves Gathered Peplum

In [39]:
import logging

def answer(prompt, similar_items_limit=10):
    print(f'Prompt: "{prompt}"\n')
    params = define_query(prompt)
    print(params)
    result = query_db(params)
    print(f"Found {len(result)} matches with Query function.\n")
    if len(result) == 0:
        result = similarity_search(prompt)
        print(f"Found {len(result)} matches with Similarity search function.\n")
        if len(result) == 0:
            return "I'm sorry, I did not find a match. Please try again with a little bit more details."
    print(f"I have found {len(result)} matching items:\n")
    similar_items = []
    for r in result:
        similar_items.extend(query_similar_items(r['id']))
        print(f"{r['name']} ({r['id']})")
    print("\n")
    if len(similar_items) > 0:
        print("Similar items that might interest you:\n")
        for i in similar_items[:similar_items_limit]:
            print(f"{i['name']} ({i['id']})")
    print("\n\n\n")
    return result